# Code from tutorial vqe-spin-sectors

##  Build hamiltonian (H2)

In [2]:
# IMPORTS
from pennylane import numpy as np
import pennylane as qml

In [3]:
symbols = ["H", "H"]
coordinates = np.array([0.0, 0.0, -0.6614, 0.0, 0.0, 0.6614])

In [4]:
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates)

print("Number of qubits = ", qubits)
print("The Hamiltonian is ", H)

Number of qubits =  4
The Hamiltonian is    (-0.24274501260941428) [Z3]
+ (-0.24274501260941422) [Z2]
+ (-0.04207255194743911) [I0]
+ (0.17771358229091763) [Z0]
+ (0.17771358229091766) [Z1]
+ (0.12293330449299356) [Z0 Z2]
+ (0.12293330449299356) [Z1 Z3]
+ (0.1676833885560135) [Z0 Z3]
+ (0.1676833885560135) [Z1 Z2]
+ (0.17059759276836797) [Z0 Z1]
+ (0.1762766139418181) [Z2 Z3]
+ (-0.04475008406301993) [Y0 Y1 X2 X3]
+ (-0.04475008406301993) [X0 X1 Y2 Y3]
+ (0.04475008406301993) [Y0 X1 X2 Y3]
+ (0.04475008406301993) [X0 Y1 Y2 X3]


## Build S2

In [5]:
electrons = 2
S2 = qml.qchem.spin2(electrons, qubits)
print(S2)

  (0.375) [Z0]
+ (0.375) [Z1]
+ (0.375) [Z2]
+ (0.375) [Z3]
+ (0.75) [I0]
+ (-0.375) [Z0 Z1]
+ (-0.375) [Z2 Z3]
+ (-0.125) [Z0 Z3]
+ (-0.125) [Z1 Z2]
+ (0.125) [Z0 Z2]
+ (0.125) [Z1 Z3]
+ (-0.125) [Y0 X1 X2 Y3]
+ (-0.125) [X0 Y1 Y2 X3]
+ (0.125) [Y0 X1 Y2 X3]
+ (0.125) [Y0 Y1 X2 X3]
+ (0.125) [Y0 Y1 Y2 Y3]
+ (0.125) [X0 X1 X2 X3]
+ (0.125) [X0 X1 Y2 Y3]
+ (0.125) [X0 Y1 X2 Y3]


##  Get initial HF-state

In [6]:
hf = qml.qchem.hf_state(electrons, qubits)
print(hf)

[1 1 0 0]


In [7]:
singles, doubles = qml.qchem.excitations(electrons, qubits, delta_sz=0)
print(singles)
print(doubles)

[[0, 2], [1, 3]]
[[0, 1, 2, 3]]


In [8]:
def circuit(params, wires):
    qml.AllSinglesDoubles(params, wires, hf, singles, doubles)

In [9]:
#dev = qml.device("lightning.qubit", wires=qubits)
dev = qml.device("default.qubit", wires=qubits)

In [10]:
@qml.qnode(dev, interface="autograd")
def cost_fn(params):
    circuit(params, wires=range(qubits))
    return qml.expval(H)

In [11]:
@qml.qnode(dev, interface="autograd")
def S2_exp_value(params):
    circuit(params, wires=range(qubits))
    return qml.expval(S2)

In [12]:
def total_spin(params):
    return -0.5 + np.sqrt(1 / 4 + S2_exp_value(params))

## Calculate ground-state (S0)

In [13]:
opt = qml.GradientDescentOptimizer(stepsize=0.8)
np.random.seed(0)  # for reproducibility
theta = np.random.normal(0, np.pi, len(singles) + len(doubles), requires_grad=True)
print(theta)

[5.54193389 1.25713095 3.07479606]


In [14]:
max_iterations = 100
conv_tol = 1e-06

for n in range(max_iterations):

    theta, prev_energy = opt.step_and_cost(cost_fn, theta)

    energy = cost_fn(theta)
    spin = total_spin(theta)

    conv = np.abs(energy - prev_energy)

    if n % 4 == 0:
        print(f"Step = {n}, Energy = {energy:.8f} Ha, S = {spin:.4f}")

    if conv <= conv_tol:
        break

print("\n" f"Final value of the ground-state energy = {energy:.8f} Ha")
print("\n" f"Optimal value of the circuit parameters = {theta}")

Step = 0, Energy = -0.09929557 Ha, S = 0.1014
Step = 4, Energy = -0.87153518 Ha, S = 0.0982
Step = 8, Energy = -1.11692841 Ha, S = 0.0087
Step = 12, Energy = -1.13529755 Ha, S = 0.0004
Step = 16, Energy = -1.13614888 Ha, S = 0.0000
Step = 20, Energy = -1.13618734 Ha, S = 0.0000

Final value of the ground-state energy = -1.13618832 Ha

Optimal value of the circuit parameters = [3.14350662 3.14087516 2.93185886]


## Calculate single excited state (S1)

In [15]:
singles, doubles = qml.qchem.excitations(electrons, qubits, delta_sz=1)
print(singles)
print(doubles)

[[1, 2]]
[]


In [16]:
def circuit(params, wires):
    qml.AllSinglesDoubles(params, wires, np.flip(hf), singles, doubles)

In [17]:
@qml.qnode(dev, interface="autograd")
def cost_fn(params):
    circuit(params, wires=range(qubits))
    return qml.expval(H)


@qml.qnode(dev, interface="autograd")
def S2_exp_value(params):
    circuit(params, wires=range(qubits))
    return qml.expval(S2)

In [18]:
np.random.seed(0)
theta = np.random.normal(0, np.pi, len(singles) + len(doubles), requires_grad=True)

max_iterations = 100
conv_tol = 1e-06

for n in range(max_iterations):

    theta, prev_energy = opt.step_and_cost(cost_fn, theta)

    energy = cost_fn(theta)
    spin = total_spin(theta)

    conv = np.abs(energy - prev_energy)

    if n % 4 == 0:
        print(f"Step = {n}, Energy = {energy:.8f} Ha, S = {spin:.4f}")

    if conv <= conv_tol:
        break

print("\n" f"Final value of the energy = {energy:.8f} Ha")
print("\n" f"Optimal value of the circuit parameters = {theta}")

Step = 0, Energy = 0.31463319 Ha, S = 0.3539
Step = 4, Energy = -0.38517129 Ha, S = 0.9391
Step = 8, Energy = -0.47698618 Ha, S = 0.9991
Step = 12, Energy = -0.47842743 Ha, S = 1.0000
Step = 16, Energy = -0.47844667 Ha, S = 1.0000

Final value of the energy = -0.47844667 Ha

Optimal value of the circuit parameters = [3.14259046]


## Plausibilization

According to: https://socratic.org/questions/how-can-i-calculate-the-excited-state-energy-level
We can calculate the excited state energy for hydrogen-like molecules (H, He+, Li+,..) by:
E_n = -Z^2*(13.61 eV)/n^ 2, with Z being the atomic number and n being the quantum level.

Ground state: −54.44 eV = -2.0006315033 Ha
First Excited state: −13.61 eV = -0.5001578758 Ha
-> spectral_gap = -40.83 = -1.5004736275 Ha

In [19]:
singles1, doubles1 = qml.qchem.excitations(electrons, qubits, delta_sz=0)
print(singles1)
print(doubles1)

[[0, 2], [1, 3]]
[[0, 1, 2, 3]]


In [20]:
def circuit1(params, wires):
    qml.AllSinglesDoubles(params, wires, hf, singles1, doubles1)

In [21]:
singles2, doubles2 = qml.qchem.excitations(electrons, qubits, delta_sz=1)
print(singles2)
print(doubles2)

[[1, 2]]
[]


In [22]:
def circuit(params, wires):
    qml.AllSinglesDoubles(params, wires, np.flip(hf), singles2, doubles2)

In [33]:
print(qml.draw(cost_fn, expansion_strategy=dev)(theta))

0: ─╭AllSinglesDoubles(M0)─┤ ╭<𝓗>
1: ─├AllSinglesDoubles(M0)─┤ ├<𝓗>
2: ─├AllSinglesDoubles(M0)─┤ ├<𝓗>
3: ─╰AllSinglesDoubles(M0)─┤ ╰<𝓗>

M0 = 
[3.14259046]


In [34]:
specs_func = qml.specs(cost_fn)
specs_func(theta)

{'resources': Resources(num_wires=4, num_gates=1, gate_types=defaultdict(<class 'int'>, {'AllSinglesDoubles': 1}), gate_sizes=defaultdict(<class 'int'>, {4: 1}), depth=1, shots=Shots(total_shots=None, shot_vector=())),
 'num_observables': 1,
 'num_diagonalizing_gates': 0,
 'num_trainable_params': 1,
 'num_device_wires': 4,
 'device_name': 'default.qubit',
 'expansion_strategy': 'gradient',
 'gradient_options': {},
 'interface': 'autograd',
 'diff_method': 'best',
 'gradient_fn': 'backprop'}

In [36]:
c = qml.transforms.compile(cost_fn)

In [37]:
print(qml.draw(c, expansion_strategy=dev)(theta))

0: ─╭|Ψ⟩──────────┤ ╭<𝓗>
1: ─├|Ψ⟩─╭G(3.14)─┤ ├<𝓗>
2: ─├|Ψ⟩─╰G(3.14)─┤ ├<𝓗>
3: ─╰|Ψ⟩──────────┤ ╰<𝓗>


In [39]:
clifford_c = qml.clifford_t_decomposition(cost_fn)

In [45]:
def circuit_decomp(params, wires):
    qml.AllSinglesDoubles.compute_decomposition(params, wires, np.flip(hf), singles2, doubles2)

In [48]:
@qml.qnode(dev, interface="autograd")
def cost_fn_decomp(params):
    circuit_decomp(params, wires=range(qubits))
    return qml.expval(H)
print(qml.draw(cost_fn_decomp)(theta))

0: ─╭|Ψ⟩──────────┤ ╭<𝓗>
1: ─├|Ψ⟩─╭G(3.14)─┤ ├<𝓗>
2: ─├|Ψ⟩─╰G(3.14)─┤ ├<𝓗>
3: ─╰|Ψ⟩──────────┤ ╰<𝓗>
